In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext
#Inferschema didn't work for most of these
import haversine
from haversine import haversine, Unit

In [2]:
spark = SparkSession.builder.appName("t1").getOrCreate()

In [3]:
sc = SparkContext.getOrCreate()

Question 1

In [4]:
bread = spark.read.format('csv').options(header='true',inferschema='true').load("C:/Users/poona/Downloads/BreadBasket_DMS.csv")

In [5]:
bread.createOrReplaceTempView("bread")

In [6]:
bread.show()

+----------+--------+-----------+-------------+
|      Date|    Time|Transaction|         Item|
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|          1|        Bread|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:07:57|          3|Hot chocolate|
|2016-10-30|10:07:57|          3|          Jam|
|2016-10-30|10:07:57|          3|      Cookies|
|2016-10-30|10:08:41|          4|       Muffin|
|2016-10-30|10:13:03|          5|       Coffee|
|2016-10-30|10:13:03|          5|       Pastry|
|2016-10-30|10:13:03|          5|        Bread|
|2016-10-30|10:16:55|          6|    Medialuna|
|2016-10-30|10:16:55|          6|       Pastry|
|2016-10-30|10:16:55|          6|       Muffin|
|2016-10-30|10:19:12|          7|    Medialuna|
|2016-10-30|10:19:12|          7|       Pastry|
|2016-10-30|10:19:12|          7|       Coffee|
|2016-10-30|10:19:12|          7|          Tea|
|2016-10-30|10:20:51|          8|       

In [7]:
bread2 = bread.withColumn("DT", to_timestamp(concat_ws(" ",bread["Date"],bread["Time"]))) #Makes a Datetime column

In [8]:
bread2.createOrReplaceTempView("bread2")

In [9]:
bread2.show()

+----------+--------+-----------+-------------+-------------------+
|      Date|    Time|Transaction|         Item|                 DT|
+----------+--------+-----------+-------------+-------------------+
|2016-10-30|09:58:11|          1|        Bread|2016-10-30 09:58:11|
|2016-10-30|10:05:34|          2| Scandinavian|2016-10-30 10:05:34|
|2016-10-30|10:05:34|          2| Scandinavian|2016-10-30 10:05:34|
|2016-10-30|10:07:57|          3|Hot chocolate|2016-10-30 10:07:57|
|2016-10-30|10:07:57|          3|          Jam|2016-10-30 10:07:57|
|2016-10-30|10:07:57|          3|      Cookies|2016-10-30 10:07:57|
|2016-10-30|10:08:41|          4|       Muffin|2016-10-30 10:08:41|
|2016-10-30|10:13:03|          5|       Coffee|2016-10-30 10:13:03|
|2016-10-30|10:13:03|          5|       Pastry|2016-10-30 10:13:03|
|2016-10-30|10:13:03|          5|        Bread|2016-10-30 10:13:03|
|2016-10-30|10:16:55|          6|    Medialuna|2016-10-30 10:16:55|
|2016-10-30|10:16:55|          6|       Pastry|2

In [10]:
query = "SELECT Date, Item FROM bread2 WHERE HOUR(DT)=8 AND Item != 'NONE'" #Gets time between 8 and 9 am and gets rid of NONE
F1 = spark.sql(query)

In [11]:
F1.createOrReplaceTempView("F1")

In [12]:
F1.show()

+----------+-------+
|      Date|   Item|
+----------+-------+
|2016-10-31| Coffee|
|2016-10-31|   Cake|
|2016-10-31|Tartine|
|2016-10-31|  Bread|
|2016-10-31| Coffee|
|2016-10-31|  Bread|
|2016-11-01| Coffee|
|2016-11-01| Pastry|
|2016-11-01|    Jam|
|2016-11-01|    Tea|
|2016-11-01|    Tea|
|2016-11-01|    Tea|
|2016-11-02| Coffee|
|2016-11-02| Coffee|
|2016-11-02| Pastry|
|2016-11-02| Coffee|
|2016-11-02|  Bread|
|2016-11-02|  Bread|
|2016-11-02| Coffee|
|2016-11-02| Pastry|
+----------+-------+
only showing top 20 rows



In [13]:
query = "SELECT Date, Item, Count(Item) AS Amt FROM F1 Group by Date, Item" #Gets the amount of each item on each day
F2 = spark.sql(query)

In [14]:
F2.createOrReplaceTempView("F2")

In [15]:
F2.show()

+----------+------------+---+
|      Date|        Item|Amt|
+----------+------------+---+
|2016-11-09|      Coffee|  1|
|2017-03-25|       Bread|  2|
|2016-11-03|       Bread|  3|
|2016-12-10|       Scone|  1|
|2017-01-06|       Bread|  1|
|2017-02-11|   Nomad bag|  1|
|2017-04-08|      Pastry|  1|
|2016-11-19|       Bread|  3|
|2016-12-05|       Toast|  1|
|2016-11-28|   Medialuna|  1|
|2017-02-10|       Bread|  2|
|2016-12-17|  Farm House|  1|
|2017-03-14|      Coffee|  1|
|2017-03-27|      Muffin|  1|
|2016-11-11|Scandinavian|  1|
|2016-12-12|      Pastry|  1|
|2017-03-10|       Bread|  5|
|2017-01-19|       Toast|  1|
|2017-02-25|      Pastry|  1|
|2016-11-08|      Pastry|  1|
+----------+------------+---+
only showing top 20 rows



In [16]:
query = "SELECT Date, Max(Amt) AS Amt FROM F2 Group by Date" #Gets the amount of the item that was sold the most on each day
F3 = spark.sql(query)

In [17]:
F3.createOrReplaceTempView("F3")

In [18]:
F3.show()

+----------+---+
|      Date|Amt|
+----------+---+
|2017-02-24|  1|
|2017-03-18|  6|
|2017-02-08|  1|
|2017-03-16|  4|
|2017-01-27|  3|
|2017-03-09|  2|
|2016-11-18|  6|
|2016-12-22|  1|
|2016-12-14|  3|
|2016-12-21|  6|
|2017-03-23|  2|
|2017-03-22|  1|
|2016-12-01|  1|
|2016-11-25|  2|
|2016-11-29|  3|
|2016-12-02|  4|
|2017-04-07|  4|
|2016-11-21|  2|
|2016-11-23|  1|
|2017-01-06|  2|
+----------+---+
only showing top 20 rows



In [19]:
#Will match the amount of the highest sold item on each day to the other items on those days and time to see all of the most sold items
query = "SELECT F2.Date, F2.Item, F2.Amt From F2 INNER JOIN F3 ON F2.Date=F3.Date AND F2.Amt=F3.Amt ORDER BY Date"
F4 = spark.sql(query)

In [20]:
F4.createOrReplaceTempView("F4")

In [21]:
F4.show()

+----------+---------+---+
|      Date|     Item|Amt|
+----------+---------+---+
|2016-10-31|   Coffee|  2|
|2016-10-31|    Bread|  2|
|2016-11-01|      Tea|  3|
|2016-11-02|   Coffee|  8|
|2016-11-03|   Coffee|  4|
|2016-11-04|   Coffee|  2|
|2016-11-04|    Bread|  2|
|2016-11-05|    Bread|  6|
|2016-11-07|   Pastry|  1|
|2016-11-07|   Coffee|  1|
|2016-11-08|   Pastry|  1|
|2016-11-08|    Bread|  1|
|2016-11-08|   Coffee|  1|
|2016-11-09|    Bread|  1|
|2016-11-09|   Pastry|  1|
|2016-11-09|   Coffee|  1|
|2016-11-10|   Coffee|  2|
|2016-11-11|    Bread|  6|
|2016-11-12|Medialuna|  1|
|2016-11-12|   Coffee|  1|
+----------+---------+---+
only showing top 20 rows



Question 2

In [22]:
bread = spark.read.format('csv').options(header='true',inferschema='true').load("C:/Users/poona/Downloads/BreadBasket_DMS.csv")

In [23]:
bread.createOrReplaceTempView("bread")

In [24]:
bread.show()

+----------+--------+-----------+-------------+
|      Date|    Time|Transaction|         Item|
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|          1|        Bread|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:07:57|          3|Hot chocolate|
|2016-10-30|10:07:57|          3|          Jam|
|2016-10-30|10:07:57|          3|      Cookies|
|2016-10-30|10:08:41|          4|       Muffin|
|2016-10-30|10:13:03|          5|       Coffee|
|2016-10-30|10:13:03|          5|       Pastry|
|2016-10-30|10:13:03|          5|        Bread|
|2016-10-30|10:16:55|          6|    Medialuna|
|2016-10-30|10:16:55|          6|       Pastry|
|2016-10-30|10:16:55|          6|       Muffin|
|2016-10-30|10:19:12|          7|    Medialuna|
|2016-10-30|10:19:12|          7|       Pastry|
|2016-10-30|10:19:12|          7|       Coffee|
|2016-10-30|10:19:12|          7|          Tea|
|2016-10-30|10:20:51|          8|       

In [25]:
query = "SELECT Transaction From bread WHERE Item='Brownie'" #Gets all transactions thathad brownies
F1 = spark.sql(query)

In [26]:
F1.createOrReplaceTempView("F1")

In [27]:
F1.show()

+-----------+
|Transaction|
+-----------+
|        371|
|        384|
|        385|
|        391|
|        392|
|        403|
|        419|
|        421|
|        459|
|        465|
|        476|
|        478|
|        480|
|        481|
|        492|
|        493|
|        499|
|        503|
|        507|
|        512|
+-----------+
only showing top 20 rows



In [28]:
#Gets all transactions and items that had brownies apart of it, also gets rid of NONE and brownies as items
query = "SELECT bread.* From bread INNER JOIN F1 ON bread.Transaction=F1.Transaction AND bread.Item != 'NONE' AND bread.Item != 'Brownie'"
F2 = spark.sql(query)

In [29]:
F2.createOrReplaceTempView("F2")

In [30]:
F2.show()

+----------+--------+-----------+--------------------+
|      Date|    Time|Transaction|                Item|
+----------+--------+-----------+--------------------+
|2016-11-03|10:39:12|        371|              Coffee|
|2016-11-03|10:39:12|        371|           Alfajores|
|2016-11-03|10:39:12|        371|               Fudge|
|2016-11-03|12:05:25|        384|              Coffee|
|2016-11-03|12:05:25|        384|            Sandwich|
|2016-11-03|12:07:24|        385|             Cookies|
|2016-11-03|13:02:37|        391|            Sandwich|
|2016-11-03|13:02:37|        391|              Coffee|
|2016-11-03|13:19:57|        392|              Pastry|
|2016-11-03|13:19:57|        392|            Focaccia|
|2016-11-03|13:19:57|        392|          Farm House|
|2016-11-03|14:26:27|        403|              Pastry|
|2016-11-03|14:26:27|        403|                Cake|
|2016-11-03|15:55:46|        419|              Coffee|
|2016-11-03|15:55:46|        419|           Alfajores|
|2016-11-0

There are 2 ways to go about this question, I can look at the total number of times each non-brownie item appeared and find the item or items that appear the most or I can find the item or items that has the most transactions, in this case the total amount of that item in a transaction wouldn't matter as long as it appears in a transaction.

First way is below:

In [31]:
#Finds item picked the most
query = "SELECT Item, Count(Item) AS AMT From F2 GROUP BY Item ORDER BY AMT DESC"
F3 = spark.sql(query)

In [32]:
F3.createOrReplaceTempView("F3")

In [33]:
F3.show(1)

+------+---+
|  Item|AMT|
+------+---+
|Coffee|237|
+------+---+
only showing top 1 row



Second way is below:

In [34]:
#Groups by transaction and item
query = "SELECT Transaction, Item, Count(Item) AS AMT From F2 GROUP BY Transaction, Item ORDER BY Transaction ASC"
F3 = spark.sql(query)

In [35]:
F3.createOrReplaceTempView("F3")

In [36]:
F3.show()

+-----------+--------------------+---+
|Transaction|                Item|AMT|
+-----------+--------------------+---+
|        371|               Fudge|  1|
|        371|           Alfajores|  1|
|        371|              Coffee|  1|
|        384|            Sandwich|  1|
|        384|              Coffee|  1|
|        385|             Cookies|  1|
|        391|              Coffee|  1|
|        391|            Sandwich|  1|
|        392|            Focaccia|  1|
|        392|          Farm House|  1|
|        392|              Pastry|  1|
|        403|                Cake|  1|
|        403|              Pastry|  1|
|        419|              Coffee|  1|
|        419|               Juice|  1|
|        419|           Alfajores|  1|
|        419|Ella's Kitchen Po...|  1|
|        421|             Cookies|  1|
|        421|               Juice|  1|
|        421|           Alfajores|  1|
+-----------+--------------------+---+
only showing top 20 rows



In [37]:
#Groups by transaction and item
query = "SELECT Item, Count(Item) AS AMT2 From F3 GROUP BY Item ORDER BY AMT2 DESC"
F4 = spark.sql(query)

In [38]:
F4.createOrReplaceTempView("F4")

In [39]:
F4.show(1)

+------+----+
|  Item|AMT2|
+------+----+
|Coffee| 186|
+------+----+
only showing top 1 row



Ends up being Coffee in both scenarios

Question 3

In [40]:
#This file is delimmited by semicolons not colons, but it says colons in the pdf
rests = spark.read.csv("C:/Users/poona/Downloads/Restaurants_in_Durham_County_NC.csv", sep =';', header = True)

In [41]:
rests.createOrReplaceTempView("rests")

In [42]:
rests.show()

+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----------------+------+----------------------+--------------------+
|    ID|        Premise_Name|    Premise_Address1|  Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|    Type_Description|   Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|
+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+----------------

In [43]:
rests2 = rests.withColumn("Opening_Date2", to_date(rests["Opening_Date"])) #Makes a Date column

In [44]:
rests2 = rests2.withColumn("Closing_Date2", to_date(rests["Closing_Date"])) #Makes a Date column

In [45]:
rests2.createOrReplaceTempView("rests2")

In [46]:
rests2.show()

+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----------------+------+----------------------+--------------------+-------------+-------------+
|    ID|        Premise_Name|    Premise_Address1|  Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|    Type_Description|   Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|Opening_Date2|Closing_Date2|
+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--

In [47]:
#Grabbing all of years, also noticing that there are only 2 dates in the Closing_Date column and both years from that columns(2010, 2015) are represented in the Opening_Date column so am going to ignore Closing_Date
query = "SELECT YEAR(Opening_Date2), YEAR(Closing_Date2) From rests2"
F1 = spark.sql(query)

In [48]:
F1.createOrReplaceTempView("F1")

In [49]:
F1.show()

+-------------------+-------------------+
|year(Opening_Date2)|year(Closing_Date2)|
+-------------------+-------------------+
|               1994|               null|
|               2003|               null|
|               2009|               null|
|               2012|               null|
|               2008|               null|
|               2008|               null|
|               2010|               null|
|               1996|               null|
|               1996|               null|
|               2000|               null|
|               2003|               null|
|               2001|               null|
|               1990|               null|
|               2008|               null|
|               1999|               null|
|               1996|               null|
|               1994|               null|
|               1996|               null|
|               2002|               null|
|               2016|               null|
+-------------------+-------------

In [50]:
query = "SELECT COUNT(DISTINCT year(Opening_Date2)) From rests2" #Gets the total amount of years represented and ignores null values
F2 = spark.sql(query)

In [51]:
F2.createOrReplaceTempView("F2")

In [52]:
F2.show()

+-----------------------------------+
|count(DISTINCT year(Opening_Date2))|
+-----------------------------------+
|                                 28|
+-----------------------------------+



There's another way to do this depending on what you mean by years represented, it could be the specific years that are in the set which is above, this would work if there are missing years in the data. But it could also be the difference between the max and min years in the data which is below.

In [53]:
query = "SELECT MAX(year(Opening_Date2))-MIN(year(Opening_Date2)) From rests2" #Gets the total amount of years represented and ignores null values
F3 = spark.sql(query)
F3.createOrReplaceTempView("F3")

In [54]:
F3.show()

+-----------------------------------------------------+
|(max(year(Opening_Date2)) - min(year(Opening_Date2)))|
+-----------------------------------------------------+
|                                                   27|
+-----------------------------------------------------+



They are both really close just a difference of 1

Question 4

In [55]:
#This file is delimmited by semicolons not colons, but it says colons in the pdf
rests = spark.read.csv("C:/Users/poona/Downloads/Restaurants_in_Durham_County_NC.csv", sep =';', header = True)

In [56]:
rests.createOrReplaceTempView("rests")

In [57]:
rests2 = rests.withColumn("Opening_Date2", to_date(rests["Opening_Date"])) #Makes a Date column

In [58]:
rests2 = rests2.withColumn("Closing_Date2", to_date(rests["Closing_Date"])) #Makes a Date column

In [59]:
rests2.createOrReplaceTempView("rests2")

In [60]:
rests2.show()

+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----------------+------+----------------------+--------------------+-------------+-------------+
|    ID|        Premise_Name|    Premise_Address1|  Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|    Type_Description|   Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|Opening_Date2|Closing_Date2|
+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--

In [61]:
#Grabs the type of restaurants opened during the 90's
query = "SELECT Rpt_Area_Desc, Opening_Date2 From rests2 WHERE Opening_Date2 BETWEEN '1990-01-01' and '2000-01-01' ORDER BY Opening_Date2" #Gets the total amount of years represented and ignores null values
F1 = spark.sql(query)

In [62]:
F1.createOrReplaceTempView("F1")

In [63]:
F1.show(F1.count())

+------------------+-------------+
|     Rpt_Area_Desc|Opening_Date2|
+------------------+-------------+
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|           Lodging|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|           Lodging|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|   1990-07-01|
|      Food Service|

In [64]:
query = "SELECT Rpt_Area_Desc, COUNT(Rpt_Area_Desc) From F1 GROUP BY Rpt_Area_Desc" #Gets the total amount of years represented and ignores null values
F2 = spark.sql(query)

In [65]:
F2.createOrReplaceTempView("F2")

In [66]:
F2.show()

+------------------+--------------------+
|     Rpt_Area_Desc|count(Rpt_Area_Desc)|
+------------------+--------------------+
|Bed&Breakfast Home|                   3|
|      Summer Camps|                   1|
|      Institutions|                  16|
| Local Confinement|                   2|
|  School Buildings|                  56|
|    Swimming Pools|                 256|
|          Day Care|                  58|
| Bed&Breakfast Inn|                   1|
|           Lodging|                  21|
|      Food Service|                 204|
+------------------+--------------------+



Question 5

In [67]:
popu = spark.read.format('csv').options(header='true',inferschema='true').load("C:/Users/poona/Downloads/populationbycountry19802010millions.csv")

In [68]:
popu.createOrReplaceTempView("popu")

In [69]:
#Converted numbers to integers and addes new columns
for i in popu.columns:
    if i != '_c0':
        popu=popu.withColumn('new_'+i, popu[i].cast('float'))

In [70]:
popu.createOrReplaceTempView("popu")

In [71]:
popu.show(1)

+-------------+---------+---------+---------+---------+---------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+---------+---------+---------+---------+---------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+--------+---------+---------+---------+--------+---------+---------+---------+---------+---------+---------+--------+---------+---------+---------+
|          _c0|     1980|     1981|     1982|     1983|     1984|     1985|     1986|     1987|    1988|     1989|    1990|     1991|     1992|     1993|    1994|     1995|    1996|     1997|     1998|     1999|    2000|     2001|     2002|     2003|     2004|     2005|     2006|    2007|     2008|     2009|     2010| new_1980| new_1981| new_1982| new_1983| new_1984|new_1985

In [72]:
for x in range(2010, 1980, -1): #Cycles through the years
    current_value='new_'+ str(x)
    past_value='new_'+ str(x-1)
    column = 'PC'+ '_' + str(x) + '_' + str(x-1)
    lowest = 'LowestPC'+ '_' + str(x) + '_' + str(x-1)
    F1 = spark.sql("SELECT _c0, 100.00*(({0}-{1})/{1}) AS {2} FROM popu".format(current_value,past_value,column)) #Gets Percentage change for every region
    F1.createOrReplaceTempView("F1")
    #F1.show()
    F2 = spark.sql("SELECT MIN({0}) AS {1} FROM F1".format(column, lowest)) #Gets lowest percentage change for each year
    F2.createOrReplaceTempView("F2")  
    #F2.show()
    F3 = spark.sql("SELECT _c0 AS Region,{0} FROM F1 INNER JOIN F2 ON F1.{1} = F2.{2}".format(lowest,column,lowest)) #Finds the region that matches the lowest percentage
    F3.createOrReplaceTempView("F3")  
    F3.show()  

+------------+-------------------+
|      Region| LowestPC_2010_2009|
+------------+-------------------+
|Cook Islands|-3.2013410417867276|
+------------+-------------------+

+------------+-------------------+
|      Region| LowestPC_2009_2008|
+------------+-------------------+
|Cook Islands|-3.2599857971407866|
+------------+-------------------+

+------------+------------------+
|      Region|LowestPC_2008_2007|
+------------+------------------+
|Cook Islands|-3.309695794385116|
+------------+------------------+

+------+------------------+
|Region|LowestPC_2007_2006|
+------+------------------+
| Nauru|-4.702196218285279|
+------+------------------+

+------+------------------+
|Region|LowestPC_2006_2005|
+------+------------------+
| Nauru| -4.39560981437188|
+------+------------------+

+----------+------------------+
|    Region|LowestPC_2005_2004|
+----------+------------------+
|Montserrat| -8.66935504691866|
+----------+------------------+

+--------+------------------+
|  R

Question 6

In [73]:
romeo = spark.read.text("C:/Users/poona/Downloads/romeo-juliet-pg1777.txt")

In [74]:
romeo.count()

3969

In [75]:
romeo.show()

+--------------------+
|               value|
+--------------------+
|                    |
|This Etext file i...|
|cooperation with ...|
|Future and Shakes...|
|Etexts that are N...|
|                    |
|*This Etext has c...|
|                    |
|<<THIS ELECTRONIC...|
|SHAKESPEARE IS CO...|
|PROVIDED BY PROJE...|
|MACHINE READABLE ...|
|(1) ARE FOR YOUR ...|
|DISTRIBUTED OR US...|
|DISTRIBUTION INCL...|
|TIME OR FOR MEMBE...|
|                    |
|*Project Gutenber...|
|in the presentati...|
|for your reading ...|
+--------------------+
only showing top 20 rows



In [76]:
romeo = romeo.filter(col("value") != "") #Gets rid of empty rows

In [77]:
romeo.show()

+--------------------+
|               value|
+--------------------+
|This Etext file i...|
|cooperation with ...|
|Future and Shakes...|
|Etexts that are N...|
|*This Etext has c...|
|<<THIS ELECTRONIC...|
|SHAKESPEARE IS CO...|
|PROVIDED BY PROJE...|
|MACHINE READABLE ...|
|(1) ARE FOR YOUR ...|
|DISTRIBUTED OR US...|
|DISTRIBUTION INCL...|
|TIME OR FOR MEMBE...|
|*Project Gutenber...|
|in the presentati...|
|for your reading ...|
|IS NEITHER SHAREW...|
|OF THE FUTURE CON...|
|BE MADE FOR *ANY*...|
|TO GIVE IT AWAY T...|
+--------------------+
only showing top 20 rows



In [78]:
romeo.count()

3611

In [79]:
romeo.createOrReplaceTempView("romeo")

In [80]:
romeo = romeo.withColumn("words", split("value", " ")) #Gets a list of words in each row

In [81]:
romeo.show()

+--------------------+--------------------+
|               value|               words|
+--------------------+--------------------+
|This Etext file i...|[This, Etext, fil...|
|cooperation with ...|[cooperation, wit...|
|Future and Shakes...|[Future, and, Sha...|
|Etexts that are N...|[Etexts, that, ar...|
|*This Etext has c...|[*This, Etext, ha...|
|<<THIS ELECTRONIC...|[<<THIS, ELECTRON...|
|SHAKESPEARE IS CO...|[SHAKESPEARE, IS,...|
|PROVIDED BY PROJE...|[PROVIDED, BY, PR...|
|MACHINE READABLE ...|[MACHINE, READABL...|
|(1) ARE FOR YOUR ...|[(1), ARE, FOR, Y...|
|DISTRIBUTED OR US...|[DISTRIBUTED, OR,...|
|DISTRIBUTION INCL...|[DISTRIBUTION, IN...|
|TIME OR FOR MEMBE...|[TIME, OR, FOR, M...|
|*Project Gutenber...|[*Project, Gutenb...|
|in the presentati...|[in, the, present...|
|for your reading ...|[for, your, readi...|
|IS NEITHER SHAREW...|[IS, NEITHER, SHA...|
|OF THE FUTURE CON...|[OF, THE, FUTURE,...|
|BE MADE FOR *ANY*...|[BE, MADE, FOR, *...|
|TO GIVE IT AWAY T...|[TO, GIVE,

In [82]:
romeo = romeo.withColumn("letters", (split(lower("value"), ""))) #Gets a list af characetrs in each word and lowercases them

In [83]:
romeo.show()

+--------------------+--------------------+--------------------+
|               value|               words|             letters|
+--------------------+--------------------+--------------------+
|This Etext file i...|[This, Etext, fil...|[t, h, i, s,  , e...|
|cooperation with ...|[cooperation, wit...|[c, o, o, p, e, r...|
|Future and Shakes...|[Future, and, Sha...|[f, u, t, u, r, e...|
|Etexts that are N...|[Etexts, that, ar...|[e, t, e, x, t, s...|
|*This Etext has c...|[*This, Etext, ha...|[*, t, h, i, s,  ...|
|<<THIS ELECTRONIC...|[<<THIS, ELECTRON...|[<, <, t, h, i, s...|
|SHAKESPEARE IS CO...|[SHAKESPEARE, IS,...|[s, h, a, k, e, s...|
|PROVIDED BY PROJE...|[PROVIDED, BY, PR...|[p, r, o, v, i, d...|
|MACHINE READABLE ...|[MACHINE, READABL...|[m, a, c, h, i, n...|
|(1) ARE FOR YOUR ...|[(1), ARE, FOR, Y...|[(, 1, ),  , a, r...|
|DISTRIBUTED OR US...|[DISTRIBUTED, OR,...|[d, i, s, t, r, i...|
|DISTRIBUTION INCL...|[DISTRIBUTION, IN...|[d, i, s, t, r, i...|
|TIME OR FOR MEMBE...|[TI

In [84]:
romeo = romeo.withColumn("letters",regexp_replace(col("letters"), "[^a-zA-Z0-9 ]", "")) #Gets rid of special characters except whitespaces

In [85]:
romeo.show()

+--------------------+--------------------+--------------------+
|               value|               words|             letters|
+--------------------+--------------------+--------------------+
|This Etext file i...|[This, Etext, fil...|this etext file i...|
|cooperation with ...|[cooperation, wit...|cooperation with ...|
|Future and Shakes...|[Future, and, Sha...|future and shakes...|
|Etexts that are N...|[Etexts, that, ar...|etexts that are n...|
|*This Etext has c...|[*This, Etext, ha...|this etext has ce...|
|<<THIS ELECTRONIC...|[<<THIS, ELECTRON...|this electronic v...|
|SHAKESPEARE IS CO...|[SHAKESPEARE, IS,...|shakespeare is co...|
|PROVIDED BY PROJE...|[PROVIDED, BY, PR...|provided by proje...|
|MACHINE READABLE ...|[MACHINE, READABL...|machine readable ...|
|(1) ARE FOR YOUR ...|[(1), ARE, FOR, Y...|1 are for your or...|
|DISTRIBUTED OR US...|[DISTRIBUTED, OR,...|distributed or us...|
|DISTRIBUTION INCL...|[DISTRIBUTION, IN...|distribution incl...|
|TIME OR FOR MEMBE...|[TI

In [86]:
romeo.createOrReplaceTempView("romeo")

In [87]:
#Just the column that has all of the filters implemented
query = "SELECT letters From romeo"
F1 = spark.sql(query)
F1.createOrReplaceTempView("F1")

In [88]:
F1.show()

+--------------------+
|             letters|
+--------------------+
|this etext file i...|
|cooperation with ...|
|future and shakes...|
|etexts that are n...|
|this etext has ce...|
|this electronic v...|
|shakespeare is co...|
|provided by proje...|
|machine readable ...|
|1 are for your or...|
|distributed or us...|
|distribution incl...|
|time or for membe...|
|project gutenberg...|
|in the presentati...|
|for your reading ...|
|is neither sharew...|
|of the future con...|
|be made for any a...|
|to give it away t...|
+--------------------+
only showing top 20 rows



In [89]:
F2 = F1.withColumn('word', explode(split(col('letters'), ' '))) #Gets the words by themselves that are in each line
F2.createOrReplaceTempView("F2") #Referenced: https://stackoverflow.com/questions/48927271/count-number-of-words-in-a-spark-dataframe

In [90]:
F2.show()

+--------------------+-----------+
|             letters|       word|
+--------------------+-----------+
|this etext file i...|       this|
|this etext file i...|      etext|
|this etext file i...|       file|
|this etext file i...|         is|
|this etext file i...|  presented|
|this etext file i...|         by|
|this etext file i...|    project|
|this etext file i...|  gutenberg|
|this etext file i...|         in|
|cooperation with ...|cooperation|
|cooperation with ...|       with|
|cooperation with ...|      world|
|cooperation with ...|    library|
|cooperation with ...|        inc|
|cooperation with ...|       from|
|cooperation with ...|      their|
|cooperation with ...|    library|
|cooperation with ...|         of|
|cooperation with ...|        the|
|future and shakes...|     future|
+--------------------+-----------+
only showing top 20 rows



In [91]:
F3 = F2.groupBy('word').count() #Grouping by word and coutning the occurences
F3.createOrReplaceTempView("F3")

In [92]:
F3.show(F3.count())

+--------------------+-----+
|                word|count|
+--------------------+-----+
|               those|   17|
|            carnegie|   10|
|                some|   58|
|726002026compuser...|    1|
|                chor|    2|
|                 art|   55|
|               still|   14|
|               cures|    1|
|           solemnity|    3|
|               feign|    1|
|          consortest|    1|
|             pitcher|    1|
|             murderd|    1|
|                earl|    1|
|                hope|    4|
|             speakst|    2|
|               joind|    1|
|              shroud|    3|
|              unfirm|    1|
|             embrace|    1|
|               often|    4|
|            received|    3|
|    neighbourstained|    1|
|               scorn|    5|
|         allcheering|    1|
|              weighd|    1|
|              nights|    6|
|             painted|    1|
|                bone|    2|
|            smelling|    1|
|             goodman|    1|
|             

Extra Credit

In [93]:
#This file is delimmited by semicolons not colons, but it says colons in the pdf
rests = spark.read.csv("C:/Users/poona/Downloads/Restaurants_in_Durham_County_NC.csv", sep =';', header = True)

In [94]:
rests.createOrReplaceTempView("rests")

In [95]:
rests2 = rests.withColumn("Opening_Date2", to_date(rests["Opening_Date"])) #Makes a Date column

In [96]:
rests2 = rests2.withColumn("Closing_Date2", to_date(rests["Closing_Date"])) #Makes a Date column

In [97]:
rests2.createOrReplaceTempView("rests2")

In [98]:
rests2.show()

+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----------------+------+----------------------+--------------------+-------------+-------------+
|    ID|        Premise_Name|    Premise_Address1|  Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|    Type_Description|   Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|Opening_Date2|Closing_Date2|
+------+--------------------+--------------------+------------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--

In [99]:
forc = spark.read.json("C:/Users/poona/Downloads/durham-nc-foreclosure-2006-2016.json")
forc.createOrReplaceTempView("forc")

In [100]:
forc.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           datasetid|              fields|            geometry|    record_timestamp|            recordid|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|foreclosure-2006-...|{217 E CORPORATIO...|{[-78.8922549, 36...|2017-03-06T12:41:...|629979c85b1cc68c1...|
|foreclosure-2006-...|{401 N QUEEN ST, ...|{[-78.895396, 35....|2017-03-06T12:41:...|e3cce8bbc3c9b804c...|
|foreclosure-2006-...|{403 N QUEEN ST, ...|{[-78.8950321, 35...|2017-03-06T12:41:...|311559ebfeffe7ebc...|
|foreclosure-2006-...|{918 GILBERT ST, ...|{[-78.8873774, 35...|2017-03-06T12:41:...|7ec0761bd385bab8a...|
|foreclosure-2006-...|{721 LIBERTY ST, ...|{[-78.888343, 35....|2017-03-06T12:41:...|c81ae2921ffca8125...|
|foreclosure-2006-...|{729 HOPKINS ST, ...|{[-78.888092, 35....|2017-03-06T12:41:...|ae17ea44c5918fd2d...|
|foreclosure-2006-...|{1302 E MAIN ST

In [101]:
forc

DataFrame[datasetid: string, fields: struct<address:string,geocode:array<double>,parcel_number:string,year:string>, geometry: struct<coordinates:array<double>,type:string>, record_timestamp: string, recordid: string]

In [102]:
query = "SELECT Premise_Name, geolocation From rests2 WHERE Rpt_Area_Desc = 'Food Service' AND Status = 'ACTIVE'" #Gets the total amount of years represented and ignores null values
F1 = spark.sql(query)
F1.createOrReplaceTempView("F1") #Filtering for active and food services

In [103]:
F1.show()

+--------------------+--------------------+
|        Premise_Name|         geolocation|
+--------------------+--------------------+
|    WEST 94TH ST PUB|35.9207272, -78.9...|
|BROOKDALE DURHAM IFS|36.0467802, -78.8...|
|       SMOOTHIE KING|35.9182655, -78.9...|
|HAMPTON INN & SUITES|36.0183378, -78.9...|
|KROGER R 381 MEAT...|35.9495321, -78.9...|
|LEONE INTERNATION...|35.9852771, -78.8...|
|  PIZZA HUT DELIVERY|36.0586094, -78.9...|
|       CHUBBYS TACOS|36.0094173, -78.9...|
|              SUBWAY|36.018111, -78.91...|
|DUKE UNIVERSITY W...|36.001517, -78.93...|
|DICKEYS BARBECUE PIT|35.9550003, -78.9...|
|DUKE UNIVERSITY/T...|35.9950533, -78.9...|
|                 KFC|36.0507859, -78.9...|
|DUKE UNIVERSITY/T...|35.9950533, -78.9...|
|TRIDENT PROVISION...|35.9953688, -78.9...|
|PURDUE PHARMA MAN...|36.1027227, -78.8...|
|FRESH DESIRES CAT...|35.9965963, -78.9...|
|             ZWELI'S|35.9965963, -78.9...|
|CEDARS OF CHAPEL ...|35.9094306, -79.0...|
|              MEKONG|35.9205016

In [104]:
split_F1 =  split(F1.geolocation, ',',) #Splitting on ,

In [105]:
F1 = F1.withColumn("lat", split_F1.getItem(0)) #A new column for latitude and longitude
F1 = F1.withColumn("long", split_F1.getItem(1))

In [106]:
F1.show()

+--------------------+--------------------+----------+------------+
|        Premise_Name|         geolocation|       lat|        long|
+--------------------+--------------------+----------+------------+
|    WEST 94TH ST PUB|35.9207272, -78.9...|35.9207272| -78.9573299|
|BROOKDALE DURHAM IFS|36.0467802, -78.8...|36.0467802| -78.8895483|
|       SMOOTHIE KING|35.9182655, -78.9...|35.9182655| -78.9593263|
|HAMPTON INN & SUITES|36.0183378, -78.9...|36.0183378| -78.9060312|
|KROGER R 381 MEAT...|35.9495321, -78.9...|35.9495321| -78.9211694|
|LEONE INTERNATION...|35.9852771, -78.8...|35.9852771|  -78.898944|
|  PIZZA HUT DELIVERY|36.0586094, -78.9...|36.0586094| -78.9279599|
|       CHUBBYS TACOS|36.0094173, -78.9...|36.0094173| -78.9218729|
|              SUBWAY|36.018111, -78.91...| 36.018111| -78.9102149|
|DUKE UNIVERSITY W...|36.001517, -78.93...| 36.001517| -78.9391734|
|DICKEYS BARBECUE PIT|35.9550003, -78.9...|35.9550003| -78.9931015|
|DUKE UNIVERSITY/T...|35.9950533, -78.9...|35.99

Filtering Coordinates, the data has 2 sets of coordinates and they are they reverse of each other, so I'll just calculate both of them

In [107]:
forc.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           datasetid|              fields|            geometry|    record_timestamp|            recordid|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|foreclosure-2006-...|{217 E CORPORATIO...|{[-78.8922549, 36...|2017-03-06T12:41:...|629979c85b1cc68c1...|
|foreclosure-2006-...|{401 N QUEEN ST, ...|{[-78.895396, 35....|2017-03-06T12:41:...|e3cce8bbc3c9b804c...|
|foreclosure-2006-...|{403 N QUEEN ST, ...|{[-78.8950321, 35...|2017-03-06T12:41:...|311559ebfeffe7ebc...|
|foreclosure-2006-...|{918 GILBERT ST, ...|{[-78.8873774, 35...|2017-03-06T12:41:...|7ec0761bd385bab8a...|
|foreclosure-2006-...|{721 LIBERTY ST, ...|{[-78.888343, 35....|2017-03-06T12:41:...|c81ae2921ffca8125...|
|foreclosure-2006-...|{729 HOPKINS ST, ...|{[-78.888092, 35....|2017-03-06T12:41:...|ae17ea44c5918fd2d...|
|foreclosure-2006-...|{1302 E MAIN ST

In [108]:
G1 = forc.select('geometry')

In [109]:
G1 = G1.select("geometry.coordinates")

In [110]:
G1.show()

+--------------------+
|         coordinates|
+--------------------+
|[-78.8922549, 36....|
|[-78.895396, 35.9...|
|[-78.8950321, 35....|
|[-78.8873774, 35....|
|[-78.888343, 35.9...|
|[-78.888092, 35.9...|
|[-78.886681, 35.9...|
|[-78.8806365, 35....|
|[-78.874621, 35.9...|
|[-78.869642, 35.9...|
|[-78.9041979, 35....|
|[-78.9060606, 35....|
|[-78.903483, 35.9...|
|[-78.8960511, 36....|
|[-78.890249, 35.9...|
|[-78.889404, 35.9...|
|[-78.8692657, 35....|
|[-78.868701, 35.9...|
|[-78.882454, 35.9...|
|[-78.8873413, 35....|
+--------------------+
only showing top 20 rows



In [111]:
G1 = G1.withColumn("merge",concat_ws(",",col("coordinates"))) #Splitting on ,

In [112]:
G1.show()

+--------------------+--------------------+
|         coordinates|               merge|
+--------------------+--------------------+
|[-78.8922549, 36....|-78.8922549,36.00...|
|[-78.895396, 35.9...|-78.895396,35.995797|
|[-78.8950321, 35....|-78.8950321,35.99...|
|[-78.8873774, 35....|-78.8873774,35.99...|
|[-78.888343, 35.9...|-78.888343,35.993026|
|[-78.888092, 35.9...| -78.888092,35.99217|
|[-78.886681, 35.9...|-78.886681,35.986...|
|[-78.8806365, 35....|-78.8806365,35.98...|
|[-78.874621, 35.9...|-78.874621,35.992438|
|[-78.869642, 35.9...|-78.869642,35.985903|
|[-78.9041979, 35....|-78.9041979,35.97299|
|[-78.9060606, 35....|-78.9060606,35.96...|
|[-78.903483, 35.9...|-78.903483,35.965573|
|[-78.8960511, 36....|-78.8960511,36.03...|
|[-78.890249, 35.9...|-78.890249,35.972325|
|[-78.889404, 35.9...|-78.889404,35.978108|
|[-78.8692657, 35....|-78.8692657,35.97...|
|[-78.868701, 35.9...| -78.868701,35.97492|
|[-78.882454, 35.9...|-78.882454,35.979723|
|[-78.8873413, 35....|-78.887341

In [113]:
split_G1 =  split(G1.merge, ',',)

In [114]:
G1 = G1.withColumn("glat", split_G1.getItem(0)) #Adding columg for latitude and longitude
G1 = G1.withColumn("glong", split_G1.getItem(1))

In [115]:
G1.show()

+--------------------+--------------------+-----------+----------+
|         coordinates|               merge|       glat|     glong|
+--------------------+--------------------+-----------+----------+
|[-78.8922549, 36....|-78.8922549,36.00...|-78.8922549|36.0013755|
|[-78.895396, 35.9...|-78.895396,35.995797| -78.895396| 35.995797|
|[-78.8950321, 35....|-78.8950321,35.99...|-78.8950321| 35.995413|
|[-78.8873774, 35....|-78.8873774,35.99...|-78.8873774|35.9957683|
|[-78.888343, 35.9...|-78.888343,35.993026| -78.888343| 35.993026|
|[-78.888092, 35.9...| -78.888092,35.99217| -78.888092|  35.99217|
|[-78.886681, 35.9...|-78.886681,35.986...| -78.886681|35.9865799|
|[-78.8806365, 35....|-78.8806365,35.98...|-78.8806365|35.9815968|
|[-78.874621, 35.9...|-78.874621,35.992438| -78.874621| 35.992438|
|[-78.869642, 35.9...|-78.869642,35.985903| -78.869642| 35.985903|
|[-78.9041979, 35....|-78.9041979,35.97299|-78.9041979|  35.97299|
|[-78.9060606, 35....|-78.9060606,35.96...|-78.9060606|35.9666

Searching for all forclosed areas within a mile

In [116]:
CJ = F1.crossJoin(G1) #Crossjoined the latitudes and longitudes of the active food services with the forclosures
CJ.createOrReplaceTempView("CJ")

In [117]:
CJ.show()

+--------------------+--------------------+----------+------------+--------------------+--------------------+-----------+----------+
|        Premise_Name|         geolocation|       lat|        long|         coordinates|               merge|       glat|     glong|
+--------------------+--------------------+----------+------------+--------------------+--------------------+-----------+----------+
|    WEST 94TH ST PUB|35.9207272, -78.9...|35.9207272| -78.9573299|[-78.8922549, 36....|-78.8922549,36.00...|-78.8922549|36.0013755|
|BROOKDALE DURHAM IFS|36.0467802, -78.8...|36.0467802| -78.8895483|[-78.8922549, 36....|-78.8922549,36.00...|-78.8922549|36.0013755|
|       SMOOTHIE KING|35.9182655, -78.9...|35.9182655| -78.9593263|[-78.8922549, 36....|-78.8922549,36.00...|-78.8922549|36.0013755|
|HAMPTON INN & SUITES|36.0183378, -78.9...|36.0183378| -78.9060312|[-78.8922549, 36....|-78.8922549,36.00...|-78.8922549|36.0013755|
|KROGER R 381 MEAT...|35.9495321, -78.9...|35.9495321| -78.9211694|[-

In [118]:
query = "SELECT Premise_Name, lat, long, glat, glong From CJ" #Filters out some columns
CJ1 = spark.sql(query)
CJ1.createOrReplaceTempView("CJ1")

In [119]:
CJ1.show()

+--------------------+----------+------------+-----------+----------+
|        Premise_Name|       lat|        long|       glat|     glong|
+--------------------+----------+------------+-----------+----------+
|    WEST 94TH ST PUB|35.9207272| -78.9573299|-78.8922549|36.0013755|
|BROOKDALE DURHAM IFS|36.0467802| -78.8895483|-78.8922549|36.0013755|
|       SMOOTHIE KING|35.9182655| -78.9593263|-78.8922549|36.0013755|
|HAMPTON INN & SUITES|36.0183378| -78.9060312|-78.8922549|36.0013755|
|KROGER R 381 MEAT...|35.9495321| -78.9211694|-78.8922549|36.0013755|
|LEONE INTERNATION...|35.9852771|  -78.898944|-78.8922549|36.0013755|
|  PIZZA HUT DELIVERY|36.0586094| -78.9279599|-78.8922549|36.0013755|
|       CHUBBYS TACOS|36.0094173| -78.9218729|-78.8922549|36.0013755|
|              SUBWAY| 36.018111| -78.9102149|-78.8922549|36.0013755|
|DUKE UNIVERSITY W...| 36.001517| -78.9391734|-78.8922549|36.0013755|
|DICKEYS BARBECUE PIT|35.9550003| -78.9931015|-78.8922549|36.0013755|
|DUKE UNIVERSITY/T..

In [120]:
CJ1=CJ1.withColumn('lat', CJ1.lat.cast('float')) #Change type to float
CJ1=CJ1.withColumn('long', CJ1.long.cast('float'))
CJ1=CJ1.withColumn('glat', CJ1.glat.cast('float'))
CJ1=CJ1.withColumn('glong', CJ1.glong.cast('float'))

In [121]:
def type_converter(x,y,z,w): #Created a function that will be used for UDF
    
    return haversine((x,y),(z,w), unit=Unit.MILES) #Referenced this: https://www.geeksforgeeks.org/how-to-write-spark-udf-user-defined-functions-in-python/

In [122]:
first_udf = udf(lambda l, m, o, p: type_converter(l, m, o, p)) #UDF function

In [123]:
CJ1=CJ1.withColumn("miles", first_udf('lat','long','glat','glong')) #Running function

In [124]:
CJ1.show()

+--------------------+---------+----------+---------+---------+-----------------+
|        Premise_Name|      lat|      long|     glat|    glong|            miles|
+--------------------+---------+----------+---------+---------+-----------------+
|    WEST 94TH ST PUB|35.920727| -78.95733|-78.89226|36.001377|8975.541515196755|
|BROOKDALE DURHAM IFS| 36.04678| -78.88955|-78.89226|36.001377|8983.157918829815|
|       SMOOTHIE KING|35.918266| -78.95933|-78.89226|36.001377|8975.401443172077|
|HAMPTON INN & SUITES|36.018337| -78.90603|-78.89226|36.001377| 8981.45460887779|
|KROGER R 381 MEAT...| 35.94953| -78.92117|-78.89226|36.001377|8977.018748761771|
|LEONE INTERNATION...|35.985275| -78.89894|-78.89226|36.001377|8979.140085167533|
|  PIZZA HUT DELIVERY| 36.05861| -78.92796|-78.89226|36.001377|8984.443314938531|
|       CHUBBYS TACOS| 36.00942|-78.921875|-78.89226|36.001377|8981.056576754514|
|              SUBWAY|36.018112| -78.91022|-78.89226|36.001377|8981.492852505886|
|DUKE UNIVERSITY

In [125]:
CJ1=CJ1.withColumn('miles', CJ1.miles.cast('float')) #Converting to float

In [126]:
CJ1.show()

+--------------------+---------+----------+---------+---------+--------+
|        Premise_Name|      lat|      long|     glat|    glong|   miles|
+--------------------+---------+----------+---------+---------+--------+
|    WEST 94TH ST PUB|35.920727| -78.95733|-78.89226|36.001377|8975.542|
|BROOKDALE DURHAM IFS| 36.04678| -78.88955|-78.89226|36.001377|8983.158|
|       SMOOTHIE KING|35.918266| -78.95933|-78.89226|36.001377|8975.401|
|HAMPTON INN & SUITES|36.018337| -78.90603|-78.89226|36.001377|8981.455|
|KROGER R 381 MEAT...| 35.94953| -78.92117|-78.89226|36.001377|8977.019|
|LEONE INTERNATION...|35.985275| -78.89894|-78.89226|36.001377| 8979.14|
|  PIZZA HUT DELIVERY| 36.05861| -78.92796|-78.89226|36.001377|8984.443|
|       CHUBBYS TACOS| 36.00942|-78.921875|-78.89226|36.001377|8981.057|
|              SUBWAY|36.018112| -78.91022|-78.89226|36.001377|8981.493|
|DUKE UNIVERSITY W...| 36.00152| -78.93917|-78.89226|36.001377|8980.745|
|DICKEYS BARBECUE PIT|   35.955|  -78.9931|-78.8922

In [127]:
CJ2 = CJ1.withColumn("within_a_mile", when((CJ1.miles>1),lit(0)) \
                     .otherwise(lit(1)))
CJ2.createOrReplaceTempView("CJ2") #Filtering for if it's within a mile or not

In [128]:
CJ2.show()

+--------------------+---------+----------+---------+---------+--------+-------------+
|        Premise_Name|      lat|      long|     glat|    glong|   miles|within_a_mile|
+--------------------+---------+----------+---------+---------+--------+-------------+
|    WEST 94TH ST PUB|35.920727| -78.95733|-78.89226|36.001377|8975.542|            0|
|BROOKDALE DURHAM IFS| 36.04678| -78.88955|-78.89226|36.001377|8983.158|            0|
|       SMOOTHIE KING|35.918266| -78.95933|-78.89226|36.001377|8975.401|            0|
|HAMPTON INN & SUITES|36.018337| -78.90603|-78.89226|36.001377|8981.455|            0|
|KROGER R 381 MEAT...| 35.94953| -78.92117|-78.89226|36.001377|8977.019|            0|
|LEONE INTERNATION...|35.985275| -78.89894|-78.89226|36.001377| 8979.14|            0|
|  PIZZA HUT DELIVERY| 36.05861| -78.92796|-78.89226|36.001377|8984.443|            0|
|       CHUBBYS TACOS| 36.00942|-78.921875|-78.89226|36.001377|8981.057|            0|
|              SUBWAY|36.018112| -78.91022|

In [129]:
query = "SELECT Premise_Name, COUNT(within_a_mile) From CJ2 GROUP BY Premise_Name"
CJ2 = spark.sql(query) #Getting all of the forclosures within a mile of each premise

In [130]:
CJ2.show()

+--------------------+--------------------+
|        Premise_Name|count(within_a_mile)|
+--------------------+--------------------+
|            GRILL 46|                1948|
|      EL DORADO'S #6|                1948|
|DUKE UNIVERSITY W...|                1948|
|  COMPARE FOODS DELI|                1948|
| BLU SEAFOOD AND BAR|                1948|
|    MCDONALD'S 35265|                1948|
|GSK COMMERCIAL OP...|                1948|
|         JADE BUFFET|                1948|
|W G PEARSON SCHOO...|                1948|
|     DPAC  3RD FLOOR|                1948|
|HARRIS TEETER 172...|                1948|
|BOB EVANS FARM RE...|                1948|
|HARRIS TEETER 224...|                1948|
| CHICK-FIL-A EXPRESS|                1948|
| THE BLUE NOTE GRILL|                1948|
|        WENDY'S 6316|                1948|
|               MATEO|                1948|
|NZINGA'S BREAKFAS...|                1948|
|LA SUPERIOR MEAT ...|                1948|
|         SPROUT CAFE|          

Now I'm going to do the same thing but in the reverse as the data had 2 sets of coordinates but flipped

In [131]:
CJ1=CJ1.withColumn("miles2", first_udf('lat','long','glong','glat')) #Running function

In [132]:
CJ1.show()

+--------------------+---------+----------+---------+---------+--------+------------------+
|        Premise_Name|      lat|      long|     glat|    glong|   miles|            miles2|
+--------------------+---------+----------+---------+---------+--------+------------------+
|    WEST 94TH ST PUB|35.920727| -78.95733|-78.89226|36.001377|8975.542| 6.655444237017848|
|BROOKDALE DURHAM IFS| 36.04678| -78.88955|-78.89226|36.001377|8983.158|3.1406647279048547|
|       SMOOTHIE KING|35.918266| -78.95933|-78.89226|36.001377|8975.401| 6.858939760320449|
|HAMPTON INN & SUITES|36.018337| -78.90603|-78.89226|36.001377|8981.455|1.4019968476996658|
|KROGER R 381 MEAT...| 35.94953| -78.92117|-78.89226|36.001377|8977.019| 3.930156514946534|
|LEONE INTERNATION...|35.985275| -78.89894|-78.89226|36.001377| 8979.14|1.1735902116748813|
|  PIZZA HUT DELIVERY| 36.05861| -78.92796|-78.89226|36.001377|8984.443| 4.429141710974804|
|       CHUBBYS TACOS| 36.00942|-78.921875|-78.89226|36.001377|8981.057|1.746178

In [133]:
CJ1=CJ1.withColumn('miles', CJ1.miles.cast('float')) #Converting to float

In [134]:
CJ1.show()

+--------------------+---------+----------+---------+---------+--------+------------------+
|        Premise_Name|      lat|      long|     glat|    glong|   miles|            miles2|
+--------------------+---------+----------+---------+---------+--------+------------------+
|    WEST 94TH ST PUB|35.920727| -78.95733|-78.89226|36.001377|8975.542| 6.655444237017848|
|BROOKDALE DURHAM IFS| 36.04678| -78.88955|-78.89226|36.001377|8983.158|3.1406647279048547|
|       SMOOTHIE KING|35.918266| -78.95933|-78.89226|36.001377|8975.401| 6.858939760320449|
|HAMPTON INN & SUITES|36.018337| -78.90603|-78.89226|36.001377|8981.455|1.4019968476996658|
|KROGER R 381 MEAT...| 35.94953| -78.92117|-78.89226|36.001377|8977.019| 3.930156514946534|
|LEONE INTERNATION...|35.985275| -78.89894|-78.89226|36.001377| 8979.14|1.1735902116748813|
|  PIZZA HUT DELIVERY| 36.05861| -78.92796|-78.89226|36.001377|8984.443| 4.429141710974804|
|       CHUBBYS TACOS| 36.00942|-78.921875|-78.89226|36.001377|8981.057|1.746178

In [135]:
CJ2 = CJ1.withColumn("within_a_mile", when((CJ1.miles>1),lit(0)) \
                     .otherwise(lit(1)))
CJ2.createOrReplaceTempView("CJ2") #Filtering for if it's within a mile or not

In [136]:
CJ2.show()

+--------------------+---------+----------+---------+---------+--------+------------------+-------------+
|        Premise_Name|      lat|      long|     glat|    glong|   miles|            miles2|within_a_mile|
+--------------------+---------+----------+---------+---------+--------+------------------+-------------+
|    WEST 94TH ST PUB|35.920727| -78.95733|-78.89226|36.001377|8975.542| 6.655444237017848|            0|
|BROOKDALE DURHAM IFS| 36.04678| -78.88955|-78.89226|36.001377|8983.158|3.1406647279048547|            0|
|       SMOOTHIE KING|35.918266| -78.95933|-78.89226|36.001377|8975.401| 6.858939760320449|            0|
|HAMPTON INN & SUITES|36.018337| -78.90603|-78.89226|36.001377|8981.455|1.4019968476996658|            0|
|KROGER R 381 MEAT...| 35.94953| -78.92117|-78.89226|36.001377|8977.019| 3.930156514946534|            0|
|LEONE INTERNATION...|35.985275| -78.89894|-78.89226|36.001377| 8979.14|1.1735902116748813|            0|
|  PIZZA HUT DELIVERY| 36.05861| -78.92796|-78

In [137]:
query = "SELECT Premise_Name, COUNT(within_a_mile) From CJ2 GROUP BY Premise_Name"
CJ2 = spark.sql(query) #Getting all of the forclosures within a mile of each premise

In [138]:
CJ2.show()

+--------------------+--------------------+
|        Premise_Name|count(within_a_mile)|
+--------------------+--------------------+
|            GRILL 46|                1948|
|      EL DORADO'S #6|                1948|
|DUKE UNIVERSITY W...|                1948|
|  COMPARE FOODS DELI|                1948|
| BLU SEAFOOD AND BAR|                1948|
|    MCDONALD'S 35265|                1948|
|GSK COMMERCIAL OP...|                1948|
|         JADE BUFFET|                1948|
|W G PEARSON SCHOO...|                1948|
|     DPAC  3RD FLOOR|                1948|
|HARRIS TEETER 172...|                1948|
|BOB EVANS FARM RE...|                1948|
|HARRIS TEETER 224...|                1948|
| CHICK-FIL-A EXPRESS|                1948|
| THE BLUE NOTE GRILL|                1948|
|        WENDY'S 6316|                1948|
|               MATEO|                1948|
|NZINGA'S BREAKFAS...|                1948|
|LA SUPERIOR MEAT ...|                1948|
|         SPROUT CAFE|          

And this is the results with the corodinates flipped from the forclosure data